# FABRIC Experimenters Workshop: One-Way Delay

## Experiment Description

<img src="./figs/OneWayDelay.png" width="50%"><br>



## Configure the Environment

### Set the Refresh Token

In [ ]:
import os
#Retrieve or set the refresh token (exprires 24 hours after login)
fabric_refresh_token=None
%store -r fabric_refresh_token
if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token

print("Refresh Token: {}".format(fabric_refresh_token))

### Create Slice Manager

In [ ]:
from fabrictestbed.slice_manager import SliceManager, Status

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

# Create Slice Manager
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, 
                             refresh_token=fabric_refresh_token, project_name='all', scope='all')

#Cofigure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

### Refresh the ID Token

ID Tokens exprire one hour after refresh. 

In [ ]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()
print()
    


## Build Slice Request


<img src="./figs/OneWayDelayTopo.png" width="70%"><br>



In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, LinkType, Layer

# Create Experiment Topology
experiemnt = ExperimentTopology()

slice_name="OneWayDelayDemo"

In [ ]:
uky1_demo1 = experiemnt.add_node(name='uky1-demo1', site='UKY')

uky1_demo1_capacity = Capacities()
uky1_demo1_capacity.set_fields(core=4, ram=32, disk=500)
uky1_demo1.set_properties(capacities=uky1_demo1_capacity, image_type='qcow2', image_ref='default_ubuntu_20')

uky1_demo1_nic1 = uky1_demo1.add_component(ctype=ComponentType.SharedNIC, model='ConnectX-6', name='uky1_demo1_nic1')
uky1_demo1_nic2 = uky1_demo1.add_component(ctype=ComponentType.SharedNIC, model='ConnectX-6', name='uky1_demo1_nic2')

In [ ]:
renc1_demo1 = experiemnt.add_node(name='renc1-demo1', site='RENC')

renc1_demo1_capacity = Capacities()
renc1_demo1_capacity.set_fields(core=4, ram=32, disk=500)
renc1_demo1.set_properties(capacities=renc1_demo1_capacity, image_type='qcow2', image_ref='default_ubuntu_20')

renc1_demo1_nic1  = renc1_demo1.add_component(ctype=ComponentType.SharedNIC, model='ConnectX-6', name='renc1_demo1_nic1')
renc1_demo1_nic2  = renc1_demo1.add_component(ctype=ComponentType.SharedNIC, model='ConnectX-6', name='renc1_demo1_nic2')

In [ ]:
uky2_demo1 = experiemnt.add_node(name='uky2-demo1', site='UKY')

uky2_demo1_capacity = Capacities()
uky2_demo1_capacity.set_fields(core=4, ram=32, disk=500)
uky2_demo1.set_properties(capacities=uky2_demo1_capacity, image_type='qcow2', image_ref='default_ubuntu_20')

uky2_demo1_nic  = uky2_demo1.add_component(ctype=ComponentType.SharedNIC, model='ConnectX-6', name='uky2_demo1_nic1')
uky2_demo1_nic  = uky2_demo1.add_component(ctype=ComponentType.SharedNIC, model='ConnectX-6', name='uky2_demo1_nic2')

In [ ]:
# Add a links
ping_link = experiemnt.add_link(name='ping_link', ltype=LinkType.DAC, layer=Layer.L2,
                         interfaces=[list(filter(lambda iff: (iff.name == 'renc1_demo1_nic1'+'-p1'), experiemnt.interface_list))[0],
                                     list(filter(lambda iff: (iff.name == 'uky1_demo1_nic1'+'-p1'), experiemnt.interface_list))[0]]
                         )

monitor_link1 = experiemnt.add_link(name='MonitorLink1', ltype=LinkType.DAC, layer=Layer.L2,
                         interfaces=[list(filter(lambda iff: (iff.name == 'uky2_demo1_nic1'+'-p1'), experiemnt.interface_list))[0],
                                     list(filter(lambda iff: (iff.name == 'uky1_demo1_nic2'+'-p1'), experiemnt.interface_list))[0]]
                         )

monitor_link2 = experiemnt.add_link(name='MonitorLink2', ltype=LinkType.DAC, layer=Layer.L2,
                         interfaces=[list(filter(lambda iff: (iff.name == 'uky2_demo1_nic2'+'-p1'), experiemnt.interface_list))[0],
                                     list(filter(lambda iff: (iff.name == 'renc1_demo1_nic2'+'-p1'), experiemnt.interface_list))[0]]
                         )

In [ ]:
# Generate Slice Graph
#slice_graph = t.serialize()
experiemnt.draw()

## Submit the Request

In [ ]:
slice_graph = experiemnt.serialize()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key)


print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
    slice_id=reservations[0].slice_id
else:
    print("Error Occurred: {}".format(reservations))

## Query Slices

In [ ]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
print("Slices {}".format(slices))

## Slice Status

In [ ]:
import time

slice_state=None
while slice_state != 'StableOK':
    status, slice_status = slice_manager.slice_status(slice_id=slice_id)
    slice_state = slice_status.slice_state
    print("Slice State: {}".format(slices[0].slice_state))
    time.sleep(20)

print("Response Status {}".format(status))
print("Slice Status {}".format(slice_status))

## Query Slivers

In [ ]:
status, slivers = slice_manager.slivers(slice_id=slice_id)

print("Response Status {}".format(status))
print("Slivers {}".format(reservations))

## Sliver Status

In [ ]:
for sliver in slivers:
    sliver_id=sliver.reservation_id
    status, reservation_status = slice_manager.sliver_status(slice_id=slice_id, sliver_id=sliver_id)

    print("Response Status {}".format(status))
    print("Reservation Status {}".format(reservation_status))

## Configure the Experiment Nodes

In [ ]:
script= '#!/bin/bash  \n' \
        'lspci  \n'

In [ ]:
import paramiko 
#from scp import SCPClient, SCPException

key = paramiko.RSAKey.from_private_key_file("/home/fabric/.ssh/id_rsa")
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

for sliver in slivers:
    sliver_id=sliver.reservation_id
    node_name = sliver.name
    management_ip = sliver.management_ip
    
    print("Node {0} IP {1}".format(node_name, management_ip))
    
    client.connect(management_ip,username='ubuntu',pkey = key)

    stdin, stdout, stderr = client.exec_command('curl -s http://protocols.netlab.uky.edu/~nasir/demo_setup.sh -o demo_setup.sh; chmod +x demo_setup.sh; sudo ./demo_setup.sh')
    print ('')
    print (str(stdout.read(),'utf-8').replace('\\n','\n'))

    client.close()

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))